<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/RAG_Advance_Retrieval_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RAG: Advance retrieval techniques using LangChain**

- Using LangChain - multi query retrieval, few other advance techniques from LangChain

- Vector DB: Weaviate / PineCone

- Embedding model: OpenAI Embedding: text-embedding-03

- LLM: gpt-3.5 from openai

- Domain: Education

Using Advanced Retrieval techniques like
- Multi-Query retriever
- Contextual compression
- Long-Context Reorder

This Lab focuses on implementing Retriever-Augmented Generation (RAG) system, designed for the botany domain, leverages advanced retrieval techniques facilitated by **LangChain**, employing **Weaviate** as the vector database and **GPT-3.5** for language modeling. This configuration enhances the precision and efficiency of botanical information retrieval and generation, ideal for academic research, educational content, and professional consultation.

**LangChain** serves as the orchestration framework that implements sophisticated retrieval techniques, crucial for dealing with complex botanical data. Key features include:

**Multi-Query Retriever**: This technique allows the system to handle multiple queries simultaneously or queries with multiple components, enhancing the system's ability to retrieve comprehensive information from a large dataset.
**Contextual Compression**: Here, relevant information is compressed into context-rich embeddings, preserving essential data while optimizing storage and retrieval processes.
**Long-Context Reorder**: This technique reorganizes the retrieved data to maintain coherence and relevance over extended contexts, vital for understanding complex botanical interactions and processes.
Weaviate is utilized as the vector database, storing and managing the vector embeddings generated using OpenAI's text-embedding-03 model. These embeddings encode detailed semantic relationships within botanical texts, enabling the system to retrieve highly relevant information rapidly and accurately.

**GPT-3.5** from **OpenAI** acts as the language model, synthesizing the retrieved information into coherent, contextually appropriate responses. Given GPT-3.5’s capacity for nuanced understanding and generation of text, it is especially adept at handling the detailed descriptions and complex terminology that characterize botanical information.

Overall, this RAG setup offers a powerful solution for the botany domain, enhancing the retrieval and presentation of botanical knowledge through advanced AI-driven techniques. It significantly aids in research, teaching, and practical applications by delivering accurate, contextually enriched responses swiftly, thus supporting deeper insights and learning in botany.







Necessary libraries for implementing this "RAG: Advance retrieval techniques using LangChain"

## Environment Setup

In [6]:
!pip install -q langchain langchain_openai langchain_community pypdf
!pip install -q "weaviate-client==3.*"

Function for showing retrieved document results in intuitive way

In [9]:
!wget -q https://anshupandey.blob.core.windows.net/generativeaidocs/BotanyDocs.zip
!unzip -q BotanyDocs.zip

In [1]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

We will need API keys of openai, weaviate for performing this solution, so we stored api keys in .env file

In [24]:
import os

os.environ["OPENAI_API_KEY"] = "sk-********************"
os.environ["WEAVIATE_API_KEY"] = "******************"
os.environ["WEAVIATE_URL"] = "https://testsb24-nwtfq3re.weaviate.network"

The RecursiveCharacterTextSplitter is used in Retriever-Augmented Generation (RAG) models for handling long text inputs.

Here are the main points for its use:

- Length Management
- Context Preservation
- Efficient Processing

## Data Preparation

In [26]:
# Import the RecursiveCharacterTextSplitter class.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize a RecursiveCharacterTextSplitter with specified separators and configurations.
# The text will be split into chunks around 1000 characters long, with a 20-character overlap between chunks.

character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],  # The hierarchy of separators to use for splitting.
    chunk_size=800,  # Target size of each chunk in characters.
    chunk_overlap=100)


Here we are using four pdfs related to Stock market investments and strategies. These are our external data sources on which we will implement RAG solution

PyPDFLoader class from the langchain_community.document_loaders module to load and process multiple PDF documents

Loop Through PDF Files: The code iterates over the list of PDF filenames using a for-loop.

Loading PDF Content: For each PDF file in the list, an instance of PyPDFLoader is created with the filename as an argument.

Extract and Split Text: The load_and_split method of the PyPDFLoader instance is called to load the text content from the PDF file and split it into manageable segments. The method uses a text_splitter for this purpose, which is RecursiveCharacterTextSplitter defined above cell

In [17]:
from langchain_community.document_loaders import PyPDFLoader
pdfs=["intro_botany.pdf","Basics_of_Plants.pdf"]
docs=[]
for i in pdfs:
    loader = PyPDFLoader(i)
    docs.extend(loader.load_and_split(text_splitter=character_splitter))

In [23]:
docs[0]

Document(page_content='Chapter 1\nIntroduction to the Introduction\n1.1 Plants, Botany, and Kingdoms\nBotany is the scientiﬁc study of plants and plant-like organisms. It helps us under-\nstand why plants are so vitally important to the world. Plants start the majority of\nfood and energy chains, they provide us with oxygen, food and medicine.\nPlants can be divided into two groups: plants 1andplants 2. Plants 1contain all pho-\ntosynthetic organisms which use light, H 2O, and CO 2to make organic compounds\nand O 2. Plants 1are deﬁned ecologically (based on their role in nature).\nSome plants 1can be bacteria or even animals! One example of this a green slug,\nElysia chlorotica (see Fig. 1.1). Green slugs collect chloroplasts from algae and use', metadata={'source': 'intro_botany.pdf', 'page': 0})

In [19]:
len(docs)

414

We are using Openai embeddings "text-embedding-ada-002"

## Vector Embeddings

In [27]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

We are using weaviate==3.* python v3

In [21]:
import weaviate
from langchain_community.vectorstores import Weaviate

auth_config = weaviate.auth.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY"))  # Replace w/ your Weaviate instance API key

# Instantiate the client with the auth config
client = weaviate.Client(
    url=os.getenv("WEAVIATE_URL"),  # Replace w/ your endpoint
    auth_client_secret=auth_config
)


In [28]:
vectorstore = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [35]:

def get_similiar_docs(query, k=6, score=False):
  if score:
    similar_docs = vectorstore.similarity_search_with_score(query, by_text=False,k=5)
  else:
    similar_docs = vectorstore.similarity_search_with_score(query, by_text=False,k=5)
  return similar_docs[0][0].page_content

query=input("what is you query? ")
get_similiar_docs(query)
# What is needed to understand life of plants"

what is you query? What is needed to understand life of plants


"1. Introduction to Botany:  \nBotany is the scientific study of plants, including their structure, function, growth, evolution, \nand classification. Plants are incredibly diverse, ranging from tiny microorganisms to giant trees, \nand they play a vital role in the Earth's ecosystem. Botany is an important field because it helps \nus understand how plants work, how they interact with their environment, and how they can be \nused to improve our lives.  \n2. Plant Anatomy:  \nPlant anatomy is the study of the structure of plants, including their external features, internal \nstructures, and tissues. Here are some basic terms you should know:  \n* Root: the underground part of a plant that absorbs water and minerals from the soil"

This below is the basic chaining process of langchain for RAG, we have provided two ways for Chainings


The first code snippet sets up a retrieval chain for answering questions based on context using Langchain. It involves creating a retrieval chain with a specific retriever and language model, structuring the question-answer process with a ChatPromptTemplate, and invoking the retrieval chain with a user input question.

In [36]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [37]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI()
chain = (
    {"context":vectorstore.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("All organic molecule is made of what?")

'All organic molecules are made of a carbon skeleton.'

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

The second code snippet constructs a chain for answering questions based on a provided context. It involves defining a template for structuring the question-answer process, creating a ChatPromptTemplate from the template, initializing a language model, and setting up a chain with components like the context (retriever), a placeholder for the question, a prompt template, the language model, and an output parser.

In [38]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Create a retrieval chain
retriever =vectorstore.as_retriever()  # Define the retriever based on your specific requirements
llm = ChatOpenAI()  # Initialize the language model


prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Invoke the retrieval chain with the question
ans=retrieval_chain.invoke({"input": input()}) # What is needed to understand life of plants
ans

What is needed to understand life of plants


{'input': 'What is needed to understand life of plants',
 'context': [Document(page_content="1. Introduction to Botany:  \nBotany is the scientific study of plants, including their structure, function, growth, evolution, \nand classification. Plants are incredibly diverse, ranging from tiny microorganisms to giant trees, \nand they play a vital role in the Earth's ecosystem. Botany is an important field because it helps \nus understand how plants work, how they interact with their environment, and how they can be \nused to improve our lives.  \n2. Plant Anatomy:  \nPlant anatomy is the study of the structure of plants, including their external features, internal \nstructures, and tissues. Here are some basic terms you should know:  \n* Root: the underground part of a plant that absorbs water and minerals from the soil", metadata={'page': 0, 'source': 'Basics_of_Plants.pdf'}),
  Document(page_content="and they play a vital role in the Earth's ecosystem. Botany is an important field beca

In [39]:
ans['answer']

'A basic knowledge of chemistry is needed to understand the life of plants. This includes knowledge of atoms, atomic weight, isotopes, elements, the periodic table, chemical bonds, valence, molecules, and molecular weight.'

**------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

These functions are created for showing the final cleaned results from advanced retrieval techniques

In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain

import re
def clean_and_format_answer(text):

  cleaned_text = text.strip()

  # Split into paragraphs and remove extra newlines
  paragraphs = re.split(r"\n\n|\n", text.strip())
  formatted_paragraphs = [paragraph.strip() for paragraph in paragraphs]

  # Join paragraphs back with a single newline
  answer = "".join(formatted_paragraphs)

  return answer


def answerRetriever(llm,docs,question):
    prompt = ChatPromptTemplate.from_template(
        """Answer the following question based only on the provided context:
        <context>
        {context}
        </context>
        Question: {input}"""
    )
    document_chain = create_stuff_documents_chain(llm, prompt)

    ans=document_chain.invoke({
        "context": unique_docs,
        "input":question
    })
    return ans

Now we are going to leverage some advanced retrieval techniques

# Multi-Query Retriever

The Multi-Query Retriever automates prompt tuning by using a language model to generate multiple queries from different perspectives for a given user input query. It retrieves relevant documents for each query and combines the unique results across all queries. By generating diverse perspectives on the same question, it aims to overcome limitations of distance-based retrieval and provide a richer set of results. The retriever simplifies the process of generating varied queries and retrieving a broader range of potentially relevant documents based on user input

In [41]:
from langchain.retrievers.multi_query import MultiQueryRetriever


question = "Explain the whole process of photosynthesis"

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm,

)



In [42]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [43]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you outline the entire process of photosynthesis?', '2. How does photosynthesis occur from start to finish?', '3. What are the steps involved in the process of photosynthesis?']


6

In [44]:
pretty_print_docs(unique_docs)

Document 1:

glucose and oxygen  
* Respiration: the process by which plants break down glucose and other organic compounds 
to produce energy  
* Transpiration: the process by which plants lose water through their leaves  
* Absorption: the process by which plants absorb water and minerals from the soil  
* Transport: the process by which plants move water, minerals, and sugars through their tissues  
4. Plant Life Cycles:  
Plants have complex life cycles that involve several stages. Here are the basic stages of a plant's 
life cycle:  
* Germination: the process by which a seed begins to grow and develop  
* Seedling: the early stage of a plant's growth, characterized by the development of roots, 
stems, and leaves
----------------------------------------------------------------------------------------------------
Document 2:

photosynthesis  
* Flower: a reproductive structure that produces seeds  
* Seed: a small, fertilized plant ovule that can grow into a new plant  
* Bark: the

In [45]:
answerRetriever(llm,unique_docs,question)

'Photosynthesis is the process by which plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It involves several stages, starting with the light stage where sun rays are absorbed by photosystems in the chloroplast. This leads to the segregation of ions, a difference in potentials, and the activation of a proton pump, resulting in the production of ATP and the assimilation of carbon dioxide. \n\nThe enzymatic stage of photosynthesis involves various participants including carbon dioxide, NADPH (a hydrogen carrier), ATP, ribulose biphosphate (RuBP), Rubisco enzyme, and other enzymes. In this stage, CO2 is assimilated into C6 molecules, which then break down into two C3 molecules (PGA). These PGA molecules go through a complex series of reactions, ultimately leading to the production of glucose. \n\nOverall, photosynthesis is a crucial process for plants as it allows them to create their own food (glucose) using sunlight, water, and carbon dioxide while releasing ox

# Contextual compression

Contextual compression in Langchain involves compressing retrieved documents using the context of a given query to filter out irrelevant information. It aims to return only the most relevant information by shortening document contents or removing documents altogether. This process helps optimize expensive language model calls and improve response quality by focusing on the essential content related to the query. Contextual compression involves passing queries to a base retriever, then compressing the retrieved documents using a Document Compressor to enhance the relevance of the information returned to the user

In [46]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:339: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Document 1:

To summarize, the logic of photosynthesis (Fig. 2.8) is based on a simple idea: make sugar from carbon dioxide. Imagine if we have letters “s”, “g”, “u”, and “a” and need to build the word “sugar”. Obviously, we will need two things: the letter “r” and the energy to put these letters in the right order. The same story occurs in photosynthesis: it will need hydrogen (H) which is the “absent letter” from CO 2 because sugars must contain H, O and C. NADP+/NADPH is used as hydrogen supplier, and energy is ATP which is created via proton pump, and the proton pump starts because light helps to concentrate protons in the reservoir.
----------------------------------------------------------------------------------------------------
Document 2:

* Photosynthesis: the process by which plants convert sunlight, water, and carbon dioxide into 
glucose and oxygen
----------------------------------------------------------------------------------------------------
Document 3:

Ultimately,

In [47]:

clean_and_format_answer(answerRetriever(llm,compressed_docs,question))

'Photosynthesis is the process by which plants convert sunlight, water, and carbon dioxide into glucose and oxygen. It begins with the light stage, where sunlight is absorbed by photosystems in the chloroplasts of plant cells. This leads to the segregation of ions, a difference in potentials, and the creation of a proton pump that generates ATP. ATP is essential for providing energy for the conversion of carbon dioxide into glucose.During the light stage, water is split into protons, electrons, and oxygen by photosystem II, which also produces ATP and forwards electrons to photosystem I. Photosystem I then uses these electrons to create NADPH, a hydrogen carrier. The light stage results in the accumulation of energy (ATP) and hydrogen (NADPH), with oxygen released as a byproduct.The enzymatic stage of photosynthesis takes place in the matrix (stroma) of the chloroplast and involves various participants such as carbon dioxide, NADPH, ATP, ribulose biphosphate (RuBP), and the enzyme Rubi


# Long-Context Reorder

Long-Context Reorder in Langchain is a technique used to address performance degradation when models access relevant information in the middle of long contexts. By reordering documents after retrieval, this method ensures that more relevant elements are placed at the beginning and end of the document list, while less relevant ones are in the middle. This reordering helps models avoid ignoring provided documents and enhances performance when dealing with long contexts and multiple retrieved documents. Long-Context Reorder is a valuable tool to optimize information retrieval and processing in complex language models

In [48]:
from langchain_community.document_transformers import (
    LongContextReorder,
)

In [49]:
query = "What is chlorophyll?"

# Get relevant documents ordered by relevance score
docs = retriever.get_relevant_documents(question)
docs

[Document(page_content="glucose and oxygen  \n* Respiration: the process by which plants break down glucose and other organic compounds \nto produce energy  \n* Transpiration: the process by which plants lose water through their leaves  \n* Absorption: the process by which plants absorb water and minerals from the soil  \n* Transport: the process by which plants move water, minerals, and sugars through their tissues  \n4. Plant Life Cycles:  \nPlants have complex life cycles that involve several stages. Here are the basic stages of a plant's \nlife cycle:  \n* Germination: the process by which a seed begins to grow and develop  \n* Seedling: the early stage of a plant's growth, characterized by the development of roots, \nstems, and leaves", metadata={'page': 0, 'source': 'Basics_of_Plants.pdf'}),
 Document(page_content='starch\nDNAgranum thylakoid\nmatrix (stroma)Figure 2.5. Chloroplast.\n* * *\nTo summarize, the logic of photosynthesis (Fig. 2.8) is based on a simple idea: make\nsuga

In [50]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

# Confirm that the 4 relevant documents are at beginning and end.
reordered_docs

[Document(page_content='starch\nDNAgranum thylakoid\nmatrix (stroma)Figure 2.5. Chloroplast.\n* * *\nTo summarize, the logic of photosynthesis (Fig. 2.8) is based on a simple idea: make\nsugar from carbon dioxide . Imagine if we have letters “s”, “g”, “u”, and “a” and need\nto build the word “sugar”. Obviously, we will need two things: the letter “r” and the\nenergy to put these letters in the right order. The same story occurs in photosynthe-\nsis: it will need hydrogen (H) which is the “absent letter” from CO 2because sugars\nmust contain H, O and C. NADP+/NADPH is used as hydrogen supplier, and energy\nis ATP which is created via proton pump, and the proton pump starts because light\nhelps to concentrate protons in the reservoir.\n33 Version June 7, 2021', metadata={'page': 14, 'source': 'intro_botany.pdf'}),
 Document(page_content='synthesis from ADP and P i(inorganic phosphate). On the other side of membrane,\nthese protons make water with hydroxide ions.\nIn the previous paragrap

In [51]:
# We prepare and run a custom Stuff chain with reordered docs as context.
clean_and_format_answer(answerRetriever(llm,reordered_docs,query))

'Chlorophyll is actually two photosystems: photosystem II (P680) and photosystem I (P700) found in the chloroplast. Photosystem II is more important as it splits water, makes a proton gradient, produces ATP, and forwards electrons to photosystem I. Photosystem I contains only chlorophylls and produces NADPH.'

## Thank You